In [1]:
import pandas as pd
import numpy as np

In [2]:
# Merge all year's trade dataset
import os
import glob
import pandas as pd
os.chdir("C:/Users/Piyush A/Desktop/TRAD/Project/Quality Content of Trade/Dataset/Refined Computation/Each Country Dataset/New folder/QCT_new_methodology/Datasets")

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Combine all files in the list and export as CSV
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "All country_2007-19dataset.csv", index=False)

In [5]:
# Read the data and extracting the required features
DF = pd.read_csv("All country_2007-19dataset.csv")

In [6]:
# Will be used for Iteration for different countries 
Y = DF.ReporterName.unique()

In [8]:
# Our final Result Dataframe
result = pd.DataFrame()

In [14]:
for name in Y:
    df = DF[DF.ReporterName == name]
    df1= df.loc[:,['ProductCode', 'Year', 'TradeFlowCode','TradeValue in 1000 USD','Quantity']]
    # Cleaning the data with removing undefined Quantity values
    df1.dropna(inplace = True)
    df1.reset_index(drop=True, inplace=True)

    # Product categorisation
    y = len(df1.columns)
    x = len(df1)
    df2 = df1.copy()

    unit_ratios = []
    unit_years = []
    unit_code = []
    for i in range(x-1):
        if df2.ProductCode[i]==df2.ProductCode[i+1] and (df2.Year[i] == df2.Year[i+1]):
                ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
                unit_code.append(df2.ProductCode[i])
                unit_ratios.append(ratio)
                unit_years.append(df2.Year[i])

    Base_data = pd.DataFrame(data=dict(Code=unit_code, Year = unit_years, Ratio = unit_ratios))

    # Categorisation done
    for i in range(len(Base_data)):
        if Base_data.Ratio[i] < 0.85:
            Base_data.loc[i,'Category'] = 'LVIIT'
        elif Base_data.Ratio[i] > 1.15:
            Base_data.loc[i,'Category'] = 'HVIIT'
        else:
            Base_data.loc[i,'Category'] = 'HIIT'

    # Calculating the count of each product category for particular year
    Index_categorisation = Base_data.groupby(['Category'])['Year'].value_counts().unstack().transpose()

    # Rename the columns so that we can add each product category contribution to the index
    Index_categorisation.rename(columns={'HIIT':'HIIT_count','HVIIT':'HVIIT_count','LVIIT':'LVIIT_count'}, inplace = True)

    Index_categorisation.reset_index(level=0, inplace = True)

    # Calculate the overall IIT Index
    GL_data = df2.copy()
    Index_df = pd.DataFrame()
    for i in range(len(GL_data)-1):
        if GL_data.ProductCode[i]== GL_data.ProductCode[i+1] and (GL_data.Year[i] == GL_data.Year[i+1]):
            Index_df.loc[i,'Year'] = GL_data['Year'].loc[i]
            Index_df.loc[i,'Sum'] = GL_data.loc[i,'TradeValue in 1000 USD']+GL_data.loc[i+1,'TradeValue in 1000 USD']
            Index_df.loc[i,'Diff'] = abs(GL_data.loc[i,'TradeValue in 1000 USD']-GL_data.loc[i+1,'TradeValue in 1000 USD']) 

    Index_group = Index_df.groupby(['Year']).sum()
    Index_group['Ratio']=1-(Index_group['Diff']/Index_group['Sum'])

    # IIT Index for all the years followed by compressing the levels of the headings
    Index_values = pd.DataFrame(Index_group, columns = ['Ratio'])
    Index_values.reset_index(level=0, inplace=True)

    # Merging the dataset to result the combination of count of each category with IIT value
    IIT_categorisation = pd.merge(Index_categorisation,Index_values, on=["Year"])

    IIT_categorisation["HIIT"] = (IIT_categorisation.Ratio)*(Index_categorisation.HIIT_count)/(Index_categorisation.HIIT_count+IIT_categorisation.HVIIT_count+Index_categorisation.LVIIT_count)
    IIT_categorisation["HVIIT"] = (IIT_categorisation.Ratio)*(Index_categorisation.HVIIT_count)/(Index_categorisation.HIIT_count+IIT_categorisation.HVIIT_count+Index_categorisation.LVIIT_count)
    IIT_categorisation["LVIIT"] = (IIT_categorisation.Ratio)*(Index_categorisation.LVIIT_count)/(Index_categorisation.HIIT_count+IIT_categorisation.HVIIT_count+Index_categorisation.LVIIT_count)
    IIT_categorisation["ReporterName"] = name
    
    result = pd.concat([result, IIT_categorisation])

<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: invalid value encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: invalid value encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered

<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: invalid value encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = (df2.loc[i,'TradeValue in 1000 USD']/df2.loc[i,'Quantity'])/(df2.loc[i+1,'TradeValue in 1000 USD']/df2.loc[i+1,'Quantity'])
<ipython-input-14-a4cea10986a4>:18: RuntimeWarning: invalid value encountered

In [15]:
result.to_csv("QCT_result.csv")